In [ ]:
'''
Name- Pranav Kushare
Kaggle UserName- Heisenberog_08
Email- pranav.kushare2001@gmail.com

Approach-
Aim of the hackathon was to classify the give gene expression signature in category control or perturb. I did a lot of experimentations, but its not possible to document everything so in this writeup I will try to explain the approach that gave me the best score.

1. The most crucial task was the appropriate feature/column selection out of those 54 columns. And then curating more data from the unlabelled data.
   After a lot of hit and trial I was able to choose ["characteristics_ch1","molecule_ch1","organism_ch1",'type',"title"] this columns which was relevant for our classification task. 

2. I tried going for simple approach that Is using only small labelled dataset for training and making submission. 
   As the data was very less it was not well generalized for testing data. That lead to the need of generating more data.

3.Next step was to generate more data. Larger datasets helped model to become more robust. So we had around 20k unlabelled datapoints. 
  I trained a voting ensemble classifier with 7 models- Naïve bayes, RF, DT, Adaboost, gradient boosting, xgb, mlp.   And used soft voting method (weight average of probabilities) to predict the labels for 15,000 samples.

4. So Now data size has been increased to 15,000 samples. I trained the classification model on this extended data and at the end logistic regression gave me the best accuracy.



'''

In [168]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlrw-biomedicalhackathon/sample_submission.csv
/kaggle/input/mlrw-biomedicalhackathon/data_only_test.csv
/kaggle/input/mlrw-biomedicalhackathon/unlabelled_train_data.csv
/kaggle/input/mlrw-biomedicalhackathon/labelled_train_data.csv


## Importing required libraries

In [169]:
# Used in all sections for managing data and files
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
import pickle
import re

# NTLK is used for preprocessing text. You can find out more about each module using their documentation.
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import inaugural, stopwords
from wordcloud import WordCloud, STOPWORDS

# Scikit-Learn is used for feature extraction and training a logistic regression model
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [170]:
labelled_training_data_path = '../input/mlrw-biomedicalhackathon/labelled_train_data.csv'
train_df = pd.read_csv(labelled_training_data_path)

## Exploring the data

In [171]:
train_df

,Unnamed: 0,geo_accession,gse_id,ctrl,pert,channel_count,characteristics_ch1,contact_address,contact_city,contact_country,...,extract_protocol_ch2,label_ch2,label_protocol_ch2,molecule_ch2,organism_ch2,source_name_ch2,taxid_ch2,treatment_protocol_ch2,biomaterial_provider_ch2,growth_protocol_ch2
0,0,GSM1617977,GSE66250,0.0,1.0,1,facs sorting: CD44low/CD24high,Am Hubland,Wuerzburg,Germany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,GSM1617983,GSE66250,0.0,1.0,1,facs sorting: Unsorted,Am Hubland,Wuerzburg,Germany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,GSM1617982,GSE66250,1.0,0.0,1,facs sorting: CD44low/CD24high,Am Hubland,Wuerzburg,Germany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,GSM1617975,GSE66250,0.0,1.0,1,facs sorting: CD44high/CD24low,Am Hubland,Wuerzburg,Germany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,GSM1267968,GSE52505,0.0,1.0,1,tissue: human nasal polyp,"148, Gurodong-ro, Guro-gu",Seoul,South Korea,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,0,GSM1462977,GSE59980,0.0,1.0,1,cell line: MCF7,1450 Biggy Street NRT6514,Los Angeles,USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
619,1,GSM1462972,GSE59980,1.0,0.0,1,cell line: MCF7,1450 Biggy Street NRT6514,Los Angeles,USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
620,2,GSM1462974,GSE59980,1.0,0.0,1,cell line: MCF7,1450 Biggy Street NRT6514,Los Angeles,USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
621,3,GSM1462976,GSE59980,0.0,1.0,1,cell line: MCF7,1450 Biggy Street NRT6514,Los Angeles,USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623 entries, 0 to 622
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                623 non-null    int64  
 1   geo_accession             623 non-null    object 
 2   gse_id                    623 non-null    object 
 3   ctrl                      623 non-null    float64
 4   pert                      623 non-null    float64
 5   channel_count             623 non-null    int64  
 6   characteristics_ch1       623 non-null    object 
 7   contact_address           623 non-null    object 
 8   contact_city              623 non-null    object 
 9   contact_country           623 non-null    object 
 10  contact_department        503 non-null    object 
 11  contact_email             394 non-null    object 
 12  contact_institute         623 non-null    object 
 13  contact_name              623 non-null    object 
 14  contact_st

## Feature seletction
#### Finding the appropriate,useful features to achieve the best possible performance of ML algorithm

In [173]:
'''
After alot of experimentation, analysis, study following 5 features gave me the best accuracy-

["characteristics_ch1","molecule_ch1","organism_ch1",'type',"title"]
'''


cols=["characteristics_ch1","molecule_ch1","organism_ch1",'type',"title"] 

train_df.loc[:, 'feature'] = ""
# We are concatenating all the columns with a space.
train_df['feature'] = train_df.apply(lambda row: ' '.join([str(row[i]) for i in cols]), axis=1)
stop_words = set(stopwords.words('english'))


In [174]:
train_df["channel_count"].value_counts()

1    597
2     26
Name: channel_count, dtype: int64

In [175]:
train_df["feature"]

0      facs sorting: CD44low/CD24high polyA RNA Homo ...
1      facs sorting: Unsorted polyA RNA Homo sapiens ...
2      facs sorting: CD44low/CD24high polyA RNA Homo ...
3      facs sorting: CD44high/CD24low polyA RNA Homo ...
4      tissue: human nasal polyp total RNA Homo sapie...
                             ...                        
618    cell line: MCF7 total RNA Homo sapiens SRA RNA...
619    cell line: MCF7 total RNA Homo sapiens SRA RNA...
620    cell line: MCF7 total RNA Homo sapiens SRA RNA...
621    cell line: MCF7 total RNA Homo sapiens SRA RNA...
622    cell line: MCF7 total RNA Homo sapiens SRA RNA...
Name: feature, Length: 623, dtype: object

In [176]:
def preprocess(data_df):
    data_df['cleaned_feature'] = ''
    stop_words = set(stopwords.words('english'))
    wordnet_lemm = WordNetLemmatizer()
    for index, row in tqdm(data_df.iterrows(), total=data_df.shape[0]):
        sample = row['feature']
        
        pre_txt = re.sub(r"[^a-zA-Z0-9- ]", " ", sample)
        pre_txt = pre_txt.lower()
        sample_words = [wordnet_lemm.lemmatize(w) for w in pre_txt.split() if w not in stop_words and len(w)>1]
        pre_proc_ver = ' '.join(sample_words)
        data_df.loc[index, 'cleaned_feature'] = pre_proc_ver
    return data_df
        
# Cleaned Training set
cleaned_train_df = preprocess(train_df.copy())


100%|██████████| 623/623 [00:00<00:00, 2289.74it/s]


In [177]:
cleaned_train_df["cleaned_feature"][0]

'facs sorting cd44low cd24high polya rna homo sapiens sra rep1 cd44low dox'

## Feature extraction and encoding 

In [178]:
'''
I had extensive experimentation between TFIDF and bag-of-words.  
Bag of words (countvectorizer) approach gave me the best score. May be because our input features had 
large vocabulary of different/unique words that was resulting into correct predictions as TFIDF works on the
principle of frequency of words, because of the uniqueness of words the frequency was going to be 1 for most of the cases.
'''




'\nI had extensive experimentation between TFIDF and bag-of-words.  \nBag of words (countvectorizer) approach gave me the best score. May be because our input features had \nlarge vocabulary of different/unique words that was resulting into correct predictions as TFIDF works on the\nprinciple of frequency of words, because of the uniqueness of words the frequency was going to be 1 for most of the cases.\n'

#### a) Extracting features from unlabelled data

In [179]:
vect = CountVectorizer(analyzer="word", preprocessor=None, stop_words=stop_words, max_features=15000)

In [180]:

unlabelled_df=pd.read_csv("../input/mlrw-biomedicalhackathon/unlabelled_train_data.csv").sample(15000,random_state=42)

unlabelled_df['feature'] = unlabelled_df.apply(lambda row: ' '.join([str(row[i]) for i in cols]), axis=1)
cleaned_unlabelled_df = preprocess(unlabelled_df.copy())

vect=vect1.fit(cleaned_unlabelled_df['cleaned_feature'])
X_unlabelled=vect.transform(cleaned_unlabelled_df['cleaned_feature'])
X_unlabelled=X_unlabelled.toarray()

100%|██████████| 15000/15000 [00:11<00:00, 1306.20it/s]


#### b) Extracting features from labelled data

In [181]:

X_train_vect = vect.transform(cleaned_train_df['cleaned_feature'])
y_train = cleaned_train_df['ctrl'].astype(int).tolist()

X = vect1.transform(cleaned_train_df['cleaned_feature']).toarray()
Y = cleaned_train_df['ctrl'].astype(int).tolist()


### Spliting into training and testing data (labelled)

In [182]:
from sklearn.model_selection import train_test_split

X_train_vect,X_valid_vect,y_train,y_valid =train_test_split(X_train_vect,y_train,test_size=0.15,\
                                                            stratify=y_train,random_state=42)
X_train_vect=X_train_vect.toarray()
X_valid_vect=X_valid_vect.toarray()
# X_train_vect

In [183]:
X_train_vect.shape

(529, 9566)

In [184]:
'''
I wrote the following code just to see how different types of classification models are performing on 
tiny labelled data (600 rows).
'''

'\nI wrote the following code just to see how different types of classification models are performing on \ntiny labelled data (600 rows).\n'

In [185]:
# from sklearn.linear_model import LogisticRegression,LinearRegression
# from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
# from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
# from sklearn.tree import ExtraTreeClassifier

# from xgboost import XGBClassifier
# import lightgbm as lgb
# from sklearn.neural_network import MLPClassifier

# from sklearn.ensemble.bagging import BaggingClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB,BernoulliNB
# from sklearn.svm import LinearSVC
# from sklearn.metrics import f1_score,roc_auc_score
# models={
#         "Logi":LogisticRegression(max_iter=1000),
#         "SVM":LinearSVC(random_state=0),
#         "Naive bayes":BernoulliNB(),
#         "RandomForest":RandomForestClassifier(n_estimators=100,random_state=0),
#         "Decision Tree": DecisionTreeClassifier(),
#         "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=0),
#         "Extra tree":ExtraTreeClassifier(),
#         "gradient_boosting":GradientBoostingClassifier(),
#         "xgb":XGBClassifier(n_estimators=200, learning_rate=0.05),
#         "lgbm":lgb.LGBMClassifier(learning_rate=0.01,num_iterations=10000,n_estimators=100),
#         "mlp":MLPClassifier(random_state=1, max_iter=500)

#         }
# nam,scores,f1,a=[],[],[],[]
# for name,model in zip(list(models.keys()),list(models.values())):
#     model.fit(X_train_vect, y_train)
#     y_pred=model.predict(X_valid_vect)
#     nam.append(name)
#     scores.append(model.score(X_valid_vect,y_valid))
#     f1.append(f1_score(y_valid,y_pred))
#     a.append(roc_auc_score(y_valid,y_pred))

# expt=pd.DataFrame({"Model":nam,"Accuracy":scores,"F1_Score":f1,"auc":a})
# expt


# Weekly supervised learning (Data Curation)

In [186]:
X_unlabelled.shape

(15000, 9566)

### Generating Labels 
#### Training (voting classifier) model on labelled data and using that to generate labels for Unlabelled-data

In [187]:
'''
Here I have used the output of 7 ML models(voting classifier) and generated the labels 
'''
models={
        "Naive bayes":BernoulliNB(),
        "RandomForest":RandomForestClassifier(n_estimators=100,random_state=0),
        "Decision Tree": DecisionTreeClassifier(),
        "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=0),
        "gradient_boosting":GradientBoostingClassifier(),
        "xgb":XGBClassifier(n_estimators=100, learning_rate=0.05),
        "mlp":MLPClassifier(random_state=1, max_iter=500)
        }

p=[]
nam,scores,f1,a=[],[],[],[]

for name,model in zip(list(models.keys()),list(models.values())):
    model.fit(X_train_vect, y_train)
    y_pred=model.predict_proba(X_unlabelled)
    p.append(y_pred)
    y_pred=model.predict(X_valid_vect)
    nam.append(name)
    scores.append(model.score(X_valid_vect,y_valid))
    f1.append(f1_score(y_valid,y_pred))
    a.append(roc_auc_score(y_valid,y_pred))
    
expt=pd.DataFrame({"Model":nam,"Accuracy":scores,"F1_Score":f1,"auc":a})
expt
    

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:40:39] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Model,Accuracy,F1_Score,auc
0,Naive bayes,0.851064,0.840909,0.850091
1,RandomForest,0.893617,0.893617,0.894022
2,Decision Tree,0.946809,0.948454,0.947917
3,AdaBoost,0.936170,0.938776,0.937500
4,gradient_boosting,0.904255,0.901099,0.903986
5,xgb,0.861702,0.850575,0.860507
6,mlp,0.914894,0.920000,0.916667


In [188]:
pred=np.array(p)
pred=np.argmax(np.sum(pred,axis=0)/5,axis=1)
pred[:5]

array([1, 0, 1, 1, 1])

In [189]:
y_predictions=pred
len(y_predictions)

15000

In [190]:
pd.Series(y_predictions).value_counts()

0    11217
1     3783
dtype: int64

### Concatenating the labelled and unlabelled data after generating the labels 
#### So now we have around 15,000 datapoints to train the final classification model

In [191]:
X=np.concatenate([X,X_unlabelled],axis=0)
Y=np.concatenate([Y,y_predictions],axis=0)

print(f"X: {X.shape}, Y: {Y.shape}")

X: (15623, 9566), Y: (15623,)


In [192]:
X_train_vect,X_valid_vect,y_train,y_valid =train_test_split(X,Y,test_size=0.20,\
                                                            stratify=Y,random_state=42)

In [193]:
X_train_vect.shape

(12498, 9566)

## Training the final classification model
#### Experimentating with different types of models

In [194]:
'''
As ensemble models were taking too much time to train. I have commented them. You can uncomment them if you want to.
But trust me all powerfull models like randomforest, xgb, lgbm are tending to overfit and performing terrible

'''
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import f1_score,roc_auc_score
models={
        "Logi":LogisticRegression(max_iter=1000),
        "Naive bayes":BernoulliNB(),
         "SVM":LinearSVC(random_state=0),
        "Decision Tree": DecisionTreeClassifier(),
        "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=0),
#         "gradient_boosting":GradientBoostingClassifier(),
#         "xgb":XGBClassifier(n_estimators=100, learning_rate=0.05),
#         "gradient_boosting":GradientBoostingClassifier(),
        }
nam,scores,f1,a=[],[],[],[]
for name,model in zip(list(models.keys()),list(models.values())):
    model.fit(X_train_vect, y_train)
    y_pred=model.predict(X_valid_vect)
    nam.append(name)
    scores.append(model.score(X_valid_vect,y_valid))
    f1.append(f1_score(y_valid,y_pred))
    a.append(roc_auc_score(y_valid,y_pred))

expt=pd.DataFrame({"Model":nam,"Accuracy":scores,"F1_Score":f1,"auc":a})
expt



/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,Model,Accuracy,F1_Score,auc
0,Logi,0.98240,0.965646,0.970327
1,Naive bayes,0.93376,0.877151,0.923970
2,SVM,0.98656,0.974265,0.981823
3,Decision Tree,0.98464,0.970874,0.982496
4,AdaBoost,0.97344,0.947897,0.957157


### Using the best possible model on test_data for predictions

In [195]:
model=LogisticRegression(max_iter=1000)
# model=LinearSVC(random_state=0)

model.fit(X_train_vect, y_train)
print("Classification report for ctrl (1 - ctrl, 0 - pert)\n")
y_predictions = model.predict(X_valid_vect)
print(classification_report(y_valid, y_predictions))



Classification report for ctrl (1 - ctrl, 0 - pert)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2307
           1       0.99      0.94      0.97       818

    accuracy                           0.98      3125
   macro avg       0.98      0.97      0.98      3125
weighted avg       0.98      0.98      0.98      3125



In [196]:

test_df = pd.read_csv('../input/mlrw-biomedicalhackathon/data_only_test.csv')

test_df.loc[:, 'feature'] = ""
test_df['feature'] = test_df.apply(lambda row: ' '.join([str(row[i]) for i in cols]), axis=1)

cleaned_test_df = preprocess(test_df.copy())
X_test_vect = final_vect.transform(cleaned_test_df['cleaned_feature'].tolist())
X_test_vect=X_test_vect.toarray()

100%|██████████| 6070/6070 [00:03<00:00, 1844.66it/s]


In [197]:

test_predictions = model.predict(X_test_vect)
cleaned_test_df['ctrl'] = test_predictions

cleaned_test_df = cleaned_test_df[['geo_accession', 'ctrl']]

# make sure its a float!
cleaned_test_df['ctrl'] = cleaned_test_df['ctrl'].astype(np.float64)

cleaned_test_df.to_csv('submission.csv', index=False)

In [200]:
'''
Best Submission-
77.90  15k voting+logi countvect cols=["characteristics_ch1","molecule_ch1","organism_ch1",'type',"title"] 

The submission that gave me the best rank on final private LB had score of 77.90 on public LB. For the submission
I used around 15,000 samples from unalabelled data. Trained a voting classifier on labelled dataset. And
used that classifier to predict/generate the labels of unlablled data. And then finally trained a logistic-regression
model on the 15k datapoints, 

'''

'\nBest Submission-\n77.90  15k voting+logi countvect cols=["characteristics_ch1","molecule_ch1","organism_ch1",\'type\',"title"] \n\nThe submission that gave me the best rank on final private LB had score of 77.90 on public LB. For the submission\nI used around 15,000 samples from unalabelled data. Trained a voting classifier on labelled dataset. And\nused that classifier to predict/generate the labels of unlablled data. And then finally trained a logistic-regression\nmodel on the 15k datapoints, \n\n'

### Experimentation LOG

##### Format-
##### LBScore----Datapoints_used----label_Generator_Model+Final_classification_model----Preprocessing----Cols_used

In [199]:
'''
Experimentation Results-->

1] 71.136 10k DT+SVM without regex cols = ["channel_count","characteristics_ch1","label_ch1","molecule_ch1","organism_ch1",'source_name_ch1','type',"title"] 

2] 75.37 10k adabost+svm prerpocessed regex Countvect cols = ["characteristics_ch1","label_ch1","molecule_ch1","organism_ch1",'type',"title"] 

3]76.47  15k adabost+svm prerpocessed regex TFIDF  cols = ["characteristics_ch1","label_ch1","molecule_ch1","organism_ch1",'type',"title"] 

4]75.584  20k DT+svm prerpocessed regex TFIDF cols = ["characteristics_ch1","label_ch1","molecule_ch1","organism_ch1",'type',"title"] 

5]75.881  20k adabost+svm CountVect cols = ["characteristics_ch1","label_ch1","molecule_ch1","organism_ch1",'type',"title"]

6] 75.189 8k DT+svm CountVect cols=["channel_count","characteristics_ch1","molecule_ch1","organism_ch1",'source_name_ch1','type',"title"] 

7]76.67 10k  voting + SVM  cols=["characteristics_ch1","label_ch1","molecule_ch1","organism_ch1",'type',"title"] 

8]77.429 15k  voting + SVM  cols=["characteristics_ch1","molecule_ch1","organism_ch1",'type',"title"] 

9]77.90  15k voting+logi cols=["characteristics_ch1","molecule_ch1","organism_ch1",'type',"title"] 
'''

'\nExperimentation Results-->\n\n1] 71.136 10k DT+SVM without regex cols = ["channel_count","characteristics_ch1","label_ch1","molecule_ch1","organism_ch1",\'source_name_ch1\',\'type\',"title"] \n\n2] 75.37 10k adabost+svm prerpocessed regex Countvect cols = ["characteristics_ch1","label_ch1","molecule_ch1","organism_ch1",\'type\',"title"] \n\n3]76.47  15k adabost+svm prerpocessed regex TFIDF  cols = ["characteristics_ch1","label_ch1","molecule_ch1","organism_ch1",\'type\',"title"] \n\n4]75.584  20k DT+svm prerpocessed regex TFIDF cols = ["characteristics_ch1","label_ch1","molecule_ch1","organism_ch1",\'type\',"title"] \n\n5]75.881  20k adabost+svm CountVect cols = ["characteristics_ch1","label_ch1","molecule_ch1","organism_ch1",\'type\',"title"]\n\n6] 75.189 8k DT+svm CountVect cols=["channel_count","characteristics_ch1","molecule_ch1","organism_ch1",\'source_name_ch1\',\'type\',"title"] \n\n7]76.67 10k  voting + SVM  cols=["characteristics_ch1","label_ch1","molecule_ch1","organism_ch